In [1]:
import os
import copy
import datetime
import random
import time
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sweetviz as sv

from IPython.display import display

import phik

from datetime import datetime
from scipy.stats import spearmanr
from scipy import stats
from scipy.stats import shapiro, mannwhitneyu

import ydata_profiling as pdp

In [2]:
df = pd.read_csv('/Users/kolotukhin.md/Downloads/science/Data_CO_TEE-Svan.csv')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
display(df.head())
print()
print(df.info())
print()
print(df.shape)
print()
print(df.dtypes)

,№ И/Б,Пол,Возраст,Date_of_operation,Рост,Вес,Операция,ФК,CO,CO II,HR,HR II,Art mean,Art mean II,CVP,CVP II,PA mean,PA mean II,PCWP,PCWP II,PEEP,PEEP II,BSA,C.I.,C.I. II,SV,SV II,SVR,SVR II,PVR,PVR II,LVSW,LVSW II,RVSW,RVSW II,LCW,LCW II,RCW,RCW II,LVOT,LVOT II,LVOT Vmax,LVOT Vmax II,LVOT Vmean,LVOT Vmean II,LVOTmaxPG,LVOTVmaxPG II,LVOT Env.Ti,LVOT Env.Ti II,LVOT VTI,LVOT VTI II,TEEHR,TEEHR II,LVSV Dopp,LVSV Dopp II,LVCO,LVCO II
0,28137/C2023,М,1950-02-18,2023-09-29,176,90,"avr, 2 cabg",II,3.5,4.5,52.0,76,47.0,61,1.0,7,10.0,17,8.0,11,3.0,3,2.06,1.7,2.2,68.0,59,1045.0,960,44.0,106,35.8,40.2,8.3,8.0,2.2,3.7,0.5,1.0,2.1,2.1,0.46,1.10,0.30,0.70,0.85,4.82,355,207,10.7,14.4,50,74,38,51,3.34,3.76
1,28722/C2023,Ж,1948-05-21,2023-10-04,160,84,"avr, 2 cabg",III,3.3,2.8,57.0,96,76.0,68,1.0,6,22.0,19,15.0,15,2.0,2,1.87,1.8,1.5,58.0,30,1827.0,1741,170.0,111,47.7,21.4,16.4,5.2,3.4,2.6,1.0,0.7,1.8,1.8,0.90,0.68,0.63,0.45,3.21,1.83,392,362,24.7,16.2,61,59,60,39,3.66,2.30
2,28459/C2023,М,1953-08-01,2023-10-05,174,73,3 cabg,I,4.2,4.9,71.0,73,70.0,68,2.0,7,16.0,16,10.0,10,1.0,5,1.86,2.2,2.6,59.0,67,1303.0,999,114.0,98,47.9,52.7,11.2,8.2,4.0,4.5,0.9,1.1,2.4,2.3,0.33,0.58,0.21,0.34,0.42,1.33,311,303,6.5,10.4,71,74,27,43,1.91,3.20
3,29170/С2023,М,1973-03-01,2023-10-11,179,90,b_p,I,3.1,3.7,57.0,59,51.0,63,0.0,7,5.0,15,7.0,8,1.0,1,1.98,1.6,1.9,54.0,63,1324.0,1198,140.0,149,32.3,47.3,3.7,6.9,2.1,3.2,0.2,0.8,2.8,2.8,0.45,0.36,0.34,0.24,0.81,0.52,311,259,10.5,6.2,57,58,64,38,3.66,2.19
4,29390/C2023,М,1967-02-11,2023-10-12,176,90,"b_p, cabg",II,3.7,7.5,50.0,78,48.0,61,3.0,2,10.0,10,8.0,10,3.0,3,2.06,1.8,3.6,74.0,96,967.0,630,42.0,22,40.4,66.4,7.1,10.4,2.4,6.2,0.5,1.0,2.3,2.3,0.41,1.43,0.28,0.92,0.66,8.22,444,244,12.4,22.4,51,81,53,96,2.71,7.82



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 57 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   № И/Б              18 non-null     object 
 1   Пол                18 non-null     object 
 2   Возраст            18 non-null     object 
 3   Date_of_operation  18 non-null     object 
 4   Рост               18 non-null     int64  
 5   Вес                18 non-null     int64  
 6   Операция           18 non-null     object 
 7   ФК                 18 non-null     object 
 8   CO                 17 non-null     float64
 9   CO II              18 non-null     float64
 10  HR                 17 non-null     float64
 11  HR II              18 non-null     int64  
 12  Art mean           17 non-null     float64
 13  Art mean II        18 non-null     int64  
 14  CVP                17 non-null     float64
 15  CVP II             18 non-null     int64  
 16  PA mean            17 non-n

In [4]:
# Создайте словарь, в котором ключи - это текущие имена столбцов, а значения - новые имена столбцов
new_column_names = {'Пол': 'sex_short',
                    '№ И/Б': 'mrn',
                    'Возраст': 'birthday',
                    'Date_of_operation': 'date_of_operation',
                    'Рост': 'height',
                    'Вес': 'weight',
                    'Операция': 'operation',
                    'ФК': 'nyha_roman',
                    'CO': 'svan_co_before',
                    'CO II': 'svan_co_after',
                    'HR': 'hr_before',
                    'HR II': 'hr_after',
                    'Art mean': 'art_mean_before',
                    'Art mean II': 'art_mean_after',
                    'CVP': 'cvp_before',
                    'CVP II': 'cvp_after',
                    'PA mean': 'pa_mean_before',
                    'PA mean II': 'pa_mean_after',
                    'PCWP': 'svan_pcwp_before',
                    'PCWP II': 'svan_pcwp_after',
                    'PEEP': 'peep_before',
                    'PEEP II': 'peep_after',
                    'C.I.': 'svan_ci_before',
                    'C.I. II': 'svan_ci_after',
                    'BSA': 'bsa',
                    'SV': 'svan_sv_before',
                    'SV II': 'svan_sv_after',
                    'SVR': 'svan_svr_before',
                    'SVR II': 'svan_svr_after',
                    'PVR': 'svan_pvr_before',
                    'PVR II': 'svan_pvr_after',
                    'LVSW': 'svan_lvsw_before',
                    'LVSW II': 'svan_lvsw_after',
                    'RVSW': 'svan_rvsw_before',
                    'RVSW II': 'svan_rvsw_after',
                    'LCW': 'svan_lcw_before',
                    'LCW II': 'svan_lcw_after',
                    'RCW': 'svan_rcw_before',
                    'RCW II': 'svan_rcw_after',
                    'LVOT': 'dopp_lvot_before',
                    'LVOT II': 'dopp_lvot_after',
                    'LVOT Vmax': 'dopp_lvot_v_max_before',
                    'LVOT Vmax II': 'dopp_lvot_v_max_after',
                    'LVOT Vmean': 'dopp_lvot_v_mean_before',
                    'LVOT Vmean II': 'dopp_lvot_v_mean_after',
                    'LVOTmaxPG': 'dopp_lvot_max_pg_before',
                    'LVOTVmaxPG II': 'dopp_lvot_max_pg_after',
                    'LVOT Env.Ti': 'dopp_lvot_env_ti_before',
                    'LVOT Env.Ti II': 'dopp_lvot_env_ti_after',
                    'LVOT VTI': 'dopp_lvot_vti_before',
                    'LVOT VTI II': 'dopp_lvot_vti_after',
                    'TEEHR': 'dopp_hr_before',
                    'TEEHR II': 'dopp_hr_after',
                    'LVSV Dopp': 'dopp_lvsv_before',
                    ' LVSV Dopp II': 'dopp_lvsv_after',
                    'LVCO': 'dopp_lvco_before',
                    'LVCO II': 'dopp_lvco_after'
                    }

# Используйте метод rename для переименования столбцов
df.rename(columns=new_column_names, inplace=True)

In [5]:
# Преобразуйте столбец 'birthday' в формат даты
df['birthday'] = pd.to_datetime(df['birthday'], format='%Y-%m-%d')

In [6]:
# Преобразуйте столбец 'date_of_operation' в формат даты
df['date_of_operation'] = pd.to_datetime(
    df['date_of_operation'], format='%Y-%m-%d')

In [7]:
# Создаем новый столбец с разницей в годах
df['age'] = (df['date_of_operation'] - df['birthday']).dt.days // 365

In [8]:
# Функция для преобразования римских цифр в арабские
def roman_to_arabic(roman_numeral):
    roman_numerals = {'I': 1, 'II': 2, 'III': 3, 'IV': 4}
    return roman_numerals.get(roman_numeral, None)


# Создаем новый столбец "nyha" с арабскими числами
df['nyha'] = df['nyha_roman'].apply(roman_to_arabic)

In [9]:
# Создаем новый столбец 'sex' с полными названиями пола
gender_mapping = {'М': 'Male',
                  'Ж': 'Female'
                  }

df['sex'] = df['sex_short'].map(gender_mapping)

In [10]:
# Удаляем столбцы ненужные для дальнейшей работы: 'start_date', 'end_date', 'nyha_roman', 'mrn', 'sex_short'
df = df.drop(['date_of_operation',
              'birthday',
              'nyha_roman',
              'mrn',
              'sex_short'
              ], axis=1
             )

In [11]:
#df = df.drop(['operation'], axis=1)

In [12]:
#df.columns = df.columns.str.replace(r"([A-Z])", r" \1").str.lower().str.replace(' ', '_').str[1:]

display(df.head(18))
print()
print(df.info())
print()
print(df.shape)
print()
print(df.dtypes)

,height,weight,operation,svan_co_before,svan_co_after,hr_before,hr_after,art_mean_before,art_mean_after,cvp_before,cvp_after,pa_mean_before,pa_mean_after,svan_pcwp_before,svan_pcwp_after,peep_before,peep_after,bsa,svan_ci_before,svan_ci_after,svan_sv_before,svan_sv_after,svan_svr_before,svan_svr_after,svan_pvr_before,svan_pvr_after,svan_lvsw_before,svan_lvsw_after,svan_rvsw_before,svan_rvsw_after,svan_lcw_before,svan_lcw_after,svan_rcw_before,svan_rcw_after,dopp_lvot_before,dopp_lvot_after,dopp_lvot_v_max_before,dopp_lvot_v_max_after,dopp_lvot_v_mean_before,dopp_lvot_v_mean_after,dopp_lvot_max_pg_before,dopp_lvot_max_pg_after,dopp_lvot_env_ti_before,dopp_lvot_env_ti_after,dopp_lvot_vti_before,dopp_lvot_vti_after,dopp_hr_before,dopp_hr_after,dopp_lvsv_before,dopp_lvsv_after,dopp_lvco_before,dopp_lvco_after,age,nyha,sex
0,176,90,"avr, 2 cabg",3.5,4.5,52.0,76,47.0,61,1.0,7,10.0,17,8.0,11,3.0,3,2.06,1.7,2.2,68.0,59,1045.0,960,44.0,106,35.8,40.2,8.3,8.0,2.2,3.7,0.5,1.0,2.1,2.1,0.46,1.10,0.30,0.70,0.85,4.82,355,207,10.7,14.4,50,74,38,51,3.34,3.76,73,2,Male
1,160,84,"avr, 2 cabg",3.3,2.8,57.0,96,76.0,68,1.0,6,22.0,19,15.0,15,2.0,2,1.87,1.8,1.5,58.0,30,1827.0,1741,170.0,111,47.7,21.4,16.4,5.2,3.4,2.6,1.0,0.7,1.8,1.8,0.90,0.68,0.63,0.45,3.21,1.83,392,362,24.7,16.2,61,59,60,39,3.66,2.30,75,3,Female
2,174,73,3 cabg,4.2,4.9,71.0,73,70.0,68,2.0,7,16.0,16,10.0,10,1.0,5,1.86,2.2,2.6,59.0,67,1303.0,999,114.0,98,47.9,52.7,11.2,8.2,4.0,4.5,0.9,1.1,2.4,2.3,0.33,0.58,0.21,0.34,0.42,1.33,311,303,6.5,10.4,71,74,27,43,1.91,3.20,70,1,Male
3,179,90,b_p,3.1,3.7,57.0,59,51.0,63,0.0,7,5.0,15,7.0,8,1.0,1,1.98,1.6,1.9,54.0,63,1324.0,1198,140.0,149,32.3,47.3,3.7,6.9,2.1,3.2,0.2,0.8,2.8,2.8,0.45,0.36,0.34,0.24,0.81,0.52,311,259,10.5,6.2,57,58,64,38,3.66,2.19,50,1,Male
4,176,90,"b_p, cabg",3.7,7.5,50.0,78,48.0,61,3.0,2,10.0,10,8.0,10,3.0,3,2.06,1.8,3.6,74.0,96,967.0,630,42.0,22,40.4,66.4,7.1,10.4,2.4,6.2,0.5,1.0,2.3,2.3,0.41,1.43,0.28,0.92,0.66,8.22,444,244,12.4,22.4,51,81,53,96,2.71,7.82,56,2,Male
5,177,74,"mvr, rf_a",4.9,4.6,64.0,78,48.0,86,4.0,6,16.0,21,10.0,11,3.0,3,1.90,2.6,2.4,77.0,59,713.0,1383,96.0,172,39.7,60.4,12.6,12.1,3.2,5.4,1.1,1.3,2.5,2.5,0.50,0.69,0.33,0.46,1.02,1.88,325,229,10.8,10.6,52,80,54,53,2.82,4.22,34,2,Male
6,180,106,"avr, 2 cabg",5.6,5.3,58.0,74,58.0,88,3.0,7,13.0,14,8.0,7,3.0,3,2.24,2.5,2.4,89.0,72,786.0,1221,71.0,105,60.3,78.9,12.1,6.8,4.4,6.3,1.0,1.0,2.1,2.1,0.45,0.95,0.35,0.59,0.81,3.63,296,259,10.2,15.2,65,64,36,54,2.34,3.46,73,2,Male
7,167,45,"b_p, mvr",5.1,5.4,50.0,76,77.0,66,2.0,4,16.0,18,8.0,9,2.0,3,1.93,2.7,2.8,103.0,71,1166.0,923,123.0,133,96.4,54.7,19.6,13.4,5.4,4.8,1.1,1.3,2.3,2.3,0.39,0.69,0.29,0.46,0.62,1.89,355,266,10.3,12.1,50,77,42,49,2.10,3.79,64,2,Female
8,156,58,avr,4.5,3.3,44.0,87,77.0,76,8.0,5,51.0,19,25.0,9,3.0,3,1.57,2.9,2.1,102.0,38,1226.0,1730,461.0,243,72.2,34.3,59.7,7.2,4.7,3.4,3.1,0.8,2.2,2.2,0.25,0.29,0.18,0.20,0.26,0.34,399,200,7.1,3.9,42,95,26,14,1.10,1.38,67,1,Female
9,165,66,3 cabg,3.1,4.3,46.0,72,67.0,68,7.0,8,11.0,14,7.0,7,3.0,3,1.86,1.7,2.3,68.0,60,1540.0,1114,102.0,129,55.2,49.5,3.7,4.9,2.8,4.0,0.5,0.8,1.8,1.8,0.80,0.67,0.52,0.35,2.33,1.80,296,296,15.5,14.7,50,76,139,37,1.99,2.84,58,2,Female



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 55 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   height                   18 non-null     int64  
 1   weight                   18 non-null     int64  
 2   operation                18 non-null     object 
 3   svan_co_before           17 non-null     float64
 4   svan_co_after            18 non-null     float64
 5   hr_before                17 non-null     float64
 6   hr_after                 18 non-null     int64  
 7   art_mean_before          17 non-null     float64
 8   art_mean_after           18 non-null     int64  
 9   cvp_before               17 non-null     float64
 10  cvp_after                18 non-null     int64  
 11  pa_mean_before           17 non-null     float64
 12  pa_mean_after            18 non-null     int64  
 13  svan_pcwp_before         17 non-null     float64
 14  svan_pcwp_after          18

In [13]:
print()
print(df.dtypes)


height                       int64
weight                       int64
operation                   object
svan_co_before             float64
svan_co_after              float64
hr_before                  float64
hr_after                     int64
art_mean_before            float64
art_mean_after               int64
cvp_before                 float64
cvp_after                    int64
pa_mean_before             float64
pa_mean_after                int64
svan_pcwp_before           float64
svan_pcwp_after              int64
peep_before                float64
peep_after                   int64
bsa                        float64
svan_ci_before             float64
svan_ci_after              float64
svan_sv_before             float64
svan_sv_after                int64
svan_svr_before            float64
svan_svr_after               int64
svan_pvr_before            float64
svan_pvr_after               int64
svan_lvsw_before           float64
svan_lvsw_after            float64
svan_rvsw_before   

In [14]:
print(df.duplicated().sum())

0


In [15]:
pd.DataFrame(df.isna().mean()
             .to_frame(name='Пропуски')
             .query('Пропуски > 0')['Пропуски'])\
    .style.background_gradient('coolwarm')\
    .format({'Пропуски': '{:.2%}'})

,Пропуски
svan_co_before,5.56%
hr_before,5.56%
art_mean_before,5.56%
cvp_before,5.56%
pa_mean_before,5.56%
svan_pcwp_before,5.56%
peep_before,5.56%
svan_ci_before,5.56%
svan_sv_before,5.56%
svan_svr_before,5.56%


In [16]:
# Список столбцов, в которых нужно заменить пропущенные значения
columns_to_fillna = ['svan_co_before',
                     'hr_before',
                     'art_mean_before',
                     'cvp_before',
                     'pa_mean_before',
                     'svan_pcwp_before',
                     'peep_before',
                     'svan_ci_before',
                     'svan_sv_before',
                     'svan_svr_before',
                     'svan_pvr_before',
                     'svan_lvsw_before',
                     'svan_rvsw_before',
                     'svan_lcw_before',
                     'svan_rcw_before'
                     ]

# Заменяем пропущенные значения средними значениями и округляем результат
df[columns_to_fillna] = df[columns_to_fillna].apply(
    lambda x: x.fillna(x.mean())).round(2)

In [17]:
pd.DataFrame(df.isna().mean()
             .to_frame(name='Пропуски')
             .query('Пропуски > 0')['Пропуски'])\
    .style.background_gradient('coolwarm')\
    .format({'Пропуски': '{:.2%}'})

,Пропуски


In [18]:
# Выберите соответствующие столбцы
columns_to_analyze = [('svan_co_before', 'dopp_lvco_before'),
                      ('svan_co_after', 'dopp_lvco_after'),
                      ('svan_sv_before', 'dopp_lvsv_before'),
                      ('svan_sv_after', 'dopp_lvsv_after')
                      ]

In [19]:
# Столбцы для анализа на нормальность
columns_to_analyze_normal = ['svan_co_before',
                             'dopp_lvco_before',
                             'svan_co_after',
                             'dopp_lvco_after',
                             'svan_sv_before',
                             'dopp_lvsv_before',
                             'svan_sv_after',
                             'dopp_lvsv_after'
                             ]

### тест Шапиро-Уилка
(для проверки нормальности данных)

In [20]:
# Проходим по столбцам и выполняем тест Шапиро-Уилка
for column in columns_to_analyze_normal:
    stat, p_value = shapiro(df[column])
    
    print(f'Тест Шапиро-Уилка для {column}:')
    print(f'Статистика: {stat}, p-значение: {p_value}')
    
    # Проверяем, если p-значение меньше 0.05 (общепринятый уровень значимости)
    if p_value > 0.05:
        print('Результат: Данные похожи на нормальное распределение (не отвергаем H0)\n')
    else:
        print('Результат: Данные не похожи на нормальное распределение (отвергаем H0)\n')

Тест Шапиро-Уилка для svan_co_before:
Статистика: 0.9617751836776733, p-значение: 0.6362423300743103
Результат: Данные похожи на нормальное распределение (не отвергаем H0)

Тест Шапиро-Уилка для dopp_lvco_before:
Статистика: 0.892483651638031, p-значение: 0.04257601127028465
Результат: Данные не похожи на нормальное распределение (отвергаем H0)

Тест Шапиро-Уилка для svan_co_after:
Статистика: 0.9302579164505005, p-значение: 0.1962277740240097
Результат: Данные похожи на нормальное распределение (не отвергаем H0)

Тест Шапиро-Уилка для dopp_lvco_after:
Статистика: 0.8777693510055542, p-значение: 0.023948298767209053
Результат: Данные не похожи на нормальное распределение (отвергаем H0)

Тест Шапиро-Уилка для svan_sv_before:
Статистика: 0.9429197907447815, p-значение: 0.3248576521873474
Результат: Данные похожи на нормальное распределение (не отвергаем H0)

Тест Шапиро-Уилка для dopp_lvsv_before:
Статистика: 0.7809971570968628, p-значение: 0.0008288168464787304
Результат: Данные не похо

#### Результат: 
- Нормальное распределение — это такое распределение данных, когда большинство значений сконцентрировано вокруг среднего, а значения по краям распределения становятся все более редкими.

  1. Нормальное распределение у столбцов: 
 - 'svan_co_before',
 - 'svan_co_after',
 - 'svan_sv_before',
 - 'svan_sv_after',
 - 'dopp_lvsv_after'
 
  2. Ненормальное распределение у столбцов: 
 - 'dopp_lvco_before',
 - 'dopp_lvco_after',
 - 'dopp_lvsv_before',

### критерий Манна-Уитни
Для выполнения попарного сравнения независимых выборок с использованием критерия Манна-Уитни (или двухвыборочного критерия Уилкоксона)

In [ ]:
columns_pairs = [('svan_co_before', 'dopp_lvco_before'),
                 ('svan_co_after', 'dopp_lvco_after'),
                 ('svan_sv_before', 'dopp_lvsv_before'),
                 ('svan_sv_after', 'dopp_lvsv_after')]

for pair in columns_pairs:
    col1, col2 = pair
    stat, p_value = mannwhitneyu(df[col1], df[col2], alternative='two-sided')
    print(f"Comparison between {col1} and {col2}:")
    print(f"Mann-Whitney U stat: {stat}")
    print(f"P-value: {p_value}")
    print("")

### Результаты сравнения с использованием критерия Манна-Уитни (Mann-Whitney U).

- Сравнение между svan_co_before и dopp_lvco_before:
   Mann-Whitney U stat: 278.0
   P-value: 0.000257
   Интерпретация: Существует статистически значимая разница между значениями svan_co_before и dopp_lvco_before.


- Сравнение между svan_co_after и dopp_lvco_after:
   Mann-Whitney U stat: 256.0
   P-value: 0.00308
   Интерпретация: Существует статистически значимая разница между значениями svan_co_after и dopp_lvco_after.


- Сравнение между svan_sv_before и dopp_lvsv_before:
   Mann-Whitney U stat: 270.0
   P-value: 0.000665
   Интерпретация: Существует статистически значимая разница между значениями svan_sv_before и dopp_lvsv_before.


- Сравнение между svan_sv_after и dopp_lvsv_after:
   Mann-Whitney U stat: 242.5
   P-value: 0.0113
   Интерпретация: Существует статистически значимая разница между значениями svan_sv_after и dopp_lvsv_after.


##### Интерпретация:
P-значения ниже общепринятого порога 0.05 свидетельствуют о том, что можно отклонить нулевую гипотезу о том, что распределения двух выборок одинаковы. В данном контексте это означает, что методы измерения различаются, и результаты статистически значимы.

In [ ]:
# Создайте новые столбцы с разностями
for col1, col2 in columns_to_analyze:
    df[f'{col1}_{col2}_diff'] = df[col1] - df[col2]

In [ ]:
# Постройте графики Бланда-Альтмана
plt.figure(figsize=(12, 8))
for i, (col1, col2) in enumerate(columns_to_analyze, 1):
    plt.subplot(2, 2, i)
    plt.scatter((df[col1] + df[col2]) / 2, df[f'{col1}_{col2}_diff'])
    plt.axhline(df[f'{col1}_{col2}_diff'].mean(),
                color='red', linestyle='dashed', linewidth=2)
    plt.title(f'Bland-Altman Plot for {col1} and {col2}')
    plt.xlabel('Mean of Measurements')
    plt.ylabel('Difference')

plt.tight_layout()
plt.show()

In [ ]:
def bland_altman_plot(data1, data2, limits_of_agreement=None):
    """
    Создает график Бланда-Альтмана для пары методов измерения.

    Параметры:
    - data1: Первый набор данных (массив или серия)
    - data2: Второй набор данных (массив или серия)
    - limits_of_agreement: Кастомные границы разброса разностей (если None, используются автоматически)

    Возвращает:
    - График Бланда-Альтмана с точками и границами разностей.
    - Статистическая информация (средняя разность, стандартное отклонение разности).
    """
    # Рассчитываем разности между парами данных
    differences = data1 - data2
    mean_difference = np.mean(differences)
    std_difference = np.std(differences, ddof=1)

    # Рассчитываем границы разностей (если не предоставлены пользователем)
    if limits_of_agreement is None:
        limits_of_agreement = [mean_difference - 1.96 * std_difference,
                               mean_difference + 1.96 * std_difference]

    # Строим график
    plt.figure(figsize=(10, 6))
    plt.scatter(np.mean([data1, data2], axis=0),
                differences, color='blue', alpha=0.7)
    plt.axhline(mean_difference, color='red', linestyle='dashed',
                linewidth=2, label='Mean Difference')
    plt.axhline(limits_of_agreement[0], color='green',
                linestyle='dashed', linewidth=2, label='Lower Limit of Agreement')
    plt.axhline(limits_of_agreement[1], color='green',
                linestyle='dashed', linewidth=2, label='Upper Limit of Agreement')

    # Добавляем некоторые аннотации и легенду
    plt.title('Bland-Altman Plot')
    plt.xlabel('Mean of Measurements')
    plt.ylabel('Difference between Measurements')
    plt.legend()

    # Выводим статистическую информацию
    print(f"Mean Difference: {mean_difference:.2f}")
    print(f"Standard Deviation of Differences: {std_difference:.2f}")

    # Проверяем статистическую значимость различий
    t_stat, p_value = stats.ttest_1samp(differences, 0)
    print(f"T-Statistic: {t_stat:.2f}")
    print(f"P-Value: {p_value:.4f}")

    plt.show()

In [ ]:
# Анализ Бланда-Альтмана для каждой пары столбцов
bland_altman_plot(df['svan_co_before'], df['dopp_lvco_before'])

### Интерпретация результатов анализа Бланда-Альтмана:

    Mean Difference (Средняя разница): 1.19
        Это среднее значение разности между измерениями двух методов.
        Положительное значение указывает на то, что первый метод в среднем дает более высокие значения, чем второй метод.

    Standard Deviation of Differences (Стандартное отклонение разности): 1.50
        Это мера разброса или изменчивости разностей между двумя методами.
        В данном случае, стандартное отклонение относительно невелико, что может указывать на согласованность между методами.

    T-Statistic (T-статистика): 3.36
        Это статистика t-теста, которая измеряет различия между средней разностью и нулевой разностью.
        Значение 3.36 говорит о том, что различия между средними разностями статистически значимы.

    P-Value (P-значение): 0.0037
        Это вероятность того, что различия между методами случайны.
        P-значение ниже 0.05 обычно считается статистически значимым.
        В данном случае, P-значение равно 0.0037, что меньше 0.05, поэтому мы отвергаем нулевую гипотезу о том, что различия случайны.

##### Интерпретация:
Результаты указывают на статистически значимые различия между методами измерения. Среднее значение разности равное 1.19 может иметь практическое значение в контексте конкретной области исследования. Важно также учитывать, что результаты могут зависеть от выбора методов и контекста применения.

In [ ]:
# Анализ Бланда-Альтмана для каждой пары столбцов
bland_altman_plot(df['svan_co_after'], df['dopp_lvco_after'])

### Интерпретация результатов анализа Бланда-Альтмана:

    Mean Difference (Среднее различие): 1.13
        Это среднее значение разностей между двумя методами измерения. В данном контексте, это означает, что среднее отклонение между измерениями метода svan_co_before и dopp_lvco_before составляет 1.13.

    Standard Deviation of Differences (Стандартное отклонение разностей): 0.86
        Это мера вариации или разброса разностей между двумя методами. В данном случае, стандартное отклонение 0.86 говорит о том, что большинство разностей лежат в пределах ±0.86 от среднего.

    T-Statistic (t-статистика): 5.59
        T-статистика используется для проверки гипотезы о том, что среднее различие между методами равно нулю. В данном случае, значение t-статистики равно 5.59. Чем больше по модулю t-статистика, тем менее вероятно, что наблюдаемые различия являются случайными.

    P-Value (p-значение): 0.0000
        Это p-значение, которое указывает на вероятность получить такие или более выраженные различия между методами, если нулевая гипотеза (о равенстве средних) верна. В данном случае, p-значение очень мало (меньше 0.05), что говорит о статистической значимости различий.

##### Интерпретация:
Высокое среднее различие (1.13) и статистически значимая t-статистика с низким p-значением указывают на то, что существует систематическое отклонение между методами измерения svan_co_before и dopp_lvco_before. Таким образом, эти два метода не согласуются между собой, и различия между ними статистически значимы.

In [ ]:
# Анализ Бланда-Альтмана для каждой пары столбцов
bland_altman_plot(df['svan_sv_before'], df['dopp_lvsv_before'])

### Интерпретация результатов анализа Бланда-Альтмана:

    Mean Difference (Средняя разность):
        Средняя разность между измерениями метода 'svan_co_before' и 'dopp_lvco_before' составляет 19.08.

    Standard Deviation of Differences (Стандартное отклонение разности):
        Стандартное отклонение разностей между методами равно 33.47. Это показывает, насколько разнообразны разности между измерениями.

    T-Statistic (T-статистика):
        T-статистика равна 2.42. Это значение используется для проверки гипотезы о том, что средняя разность равна нулю. Чем больше T-статистика, тем больше оснований отвергнуть эту гипотезу.

    P-Value (P-значение):
        P-значение равно 0.0270. Это значение показывает вероятность получить такие или более экстремальные результаты, если нулевая гипотеза (о равенстве средних) верна. Обычно, если P-значение меньше уровня значимости (обычно 0.05), мы отвергаем нулевую гипотезу.

    Интерпретация:
        С учетом P-значения (0.0270), мы имеем статистическую значимость и можем отвергнуть гипотезу о том, что средние разности равны нулю.
        Средняя разность 19.08 может быть важной с точки зрения практической значимости, в зависимости от контекста измерений.

##### Интерпретация: 
Есть статистически значимая разница между измерениями методов 'svan_co_before' и 'dopp_lvco_before', и эта разница имеет практическую важность.

In [ ]:
# Анализ Бланда-Альтмана для каждой пары столбцов
bland_altman_plot(df['svan_sv_after'], df['dopp_lvsv_after'])

### Интерпретация результатов анализа Бланда-Альтмана:

    Mean Difference (Средняя разность): Значение 13.06 указывает на среднюю разницу между измерениями методов 'svan_co_before' и 'dopp_lvco_before'. Это может быть использовано для оценки смещения между методами.

    Standard Deviation of Differences (Стандартное отклонение разности): Значение 13.18 представляет собой стандартное отклонение разностей между методами. Он измеряет разброс данных относительно средней разности. Большое стандартное отклонение может указывать на большой разброс между измерениями.

    T-Statistic (T-статистика): Значение 4.20 - это результат t-теста для проверки гипотезы о том, что средняя разница равна нулю. Большое значение t-статистики (в сочетании с низким уровнем значимости) может свидетельствовать о статистически значимой разнице между методами измерения.

    P-Value (P-значение): Значение 0.0006 является вероятностью того, что различие между методами измерения могло произойти случайно. Если это значение меньше выбранного уровня значимости (например, 0.05), то мы можем отклонить нулевую гипотезу (о равенстве средних) и считать различие статистически значимым.

##### Интерпретация: 
Итак, в данном случае, средняя разница между методами измерения 'svan_co_before' и 'dopp_lvco_before' статистически значима, и существует статистическое подтверждение различий между этими методами.